In [9]:
import matplotlib.pyplot as plt 
import pandas as pd
import math
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GRU 

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import psycopg2
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt 
%matplotlib inline
plt.rcParams['figure.facecolor'] = 'white'
import seaborn as sns

import keras_functions
from keras_functions import *

In [2]:
con = psycopg2.connect(database='codeforces', user='Joy')
cur = con.cursor()

In [3]:
binvars = get_categorical_variables([
    'all_participanttypes',
    'all_tags',
    'all_language'
])

# Initial pipeline

In [4]:
with open('set_train.txt') as f:
    train_handles = [h.strip() for h in f.readlines()]
with open('set_test.txt') as f:
    test_handles = [h.strip() for h in f.readlines()]

In [172]:
user_batch = 10

In [242]:
for c in x[0].columns:
    print c

COMPILATION_ERROR
CONTESTANT
GNU C
GNU C++
GNU C++11
MEMORY_LIMIT_EXCEEDED
MS C++
OK
PRACTICE
RUNTIME_ERROR
TIME_LIMIT_EXCEEDED
VIRTUAL
WRONG_ANSWER
contestid
newrating
oldrating
points
problem_rating
rank
smoothed_3months
shortest paths
2-sat
greedy
meet-in-the-middle
matrices
number theory
constructive algorithms
chinese remainder theorem
implementation
ternary search
schedules
dsu
dfs and similar
graph matchings
string suffix structures
math
probabilities
fft
divide and conquer
two pointers
trees
data structures
flows
sortings
expression parsing
dp
hashing
bitmasks
*special
geometry
combinatorics
graphs
brute force
games
binary search
strings
PRESENTATION_ERROR
TESTING
CRASHED
CHALLENGED
REJECTED
FAILED
SKIPPED
IDLENESS_LIMIT_EXCEEDED
PARTIAL
GYM
OUT_OF_COMPETITION
Scala
Delphi
JavaScript
PyPy 2
PyPy 3
Perl
Factor
D
Mysterious Language
FPC
Ada
F#
Java 8
Picat
Ruby
Java 8 ZIP
Pike
MS C#
Python 2
Python 3
Secret_171
Kotlin
J
Ocaml
Java 6
Go
Roco
GNU C11
Befunge
Rust
PHP
FALSE
Mono C#


In [139]:
#np.any(np.isnan(test.delta_smoothed_3months))

AttributeError: 'list' object has no attribute 'delta_smoothed_3months'

In [253]:
reload(keras_functions)
X = []
Y = []
lens = [0] * user_batch
runsum = 0
for i in range(user_batch):
    filename = 'rnn_train/' + train_handles[i] + ".csv"
    print filename
    x, y = keras_functions.get_user_data(train_handles[i], binvars=binvars, month=3, maxtimepts=500)
    X.append(x)
    Y.append(y)
    runsum += x.shape[0]
    lens[i] = runsum

rnn_train/2difficult4me.csv
rnn_train/priyanka_1998.csv
rnn_train/vatr.csv
rnn_train/shantanushanto.csv
rnn_train/KMPArthas.csv
rnn_train/914295860.csv
rnn_train/diver_ru.csv
rnn_train/yangzx1004.csv
rnn_train/BigChampion.csv
rnn_train/lbn187.csv


In [254]:
X_batch = np.concatenate(X, axis=0)

In [256]:
X_batch.shape

(161, 500, 109)

In [118]:
Y

[array([ nan,  nan,  nan]),
 array([ nan,  nan,  nan,  nan,  nan,  nan,  nan]),
 array([-0.09   , -0.09   ,  0.21   , -0.09   , -0.1575 , -0.48   ,
         0.14   ,  0.315  ,  0.315  , -0.045  , -0.045  ,  0.10375,  0.35   ]),
 array([        nan,         nan, -0.1275    , -0.1275    , -0.14178571,
        -0.14178571, -0.3775    , -0.29928571, -0.03916667, -0.11208333,
        -0.02025   , -0.29928571, -0.04875   ,  0.07625   , -0.14178571,
        -0.08783333, -0.08783333, -0.06175   , -0.08783333, -0.11666667,
         0.223     ,  0.207     , -0.06708333, -0.06708333,  0.207     ,
         0.207     ,  0.06966667,  0.06966667, -0.29928571, -0.29928571,
         0.105     ,  0.105     ,  0.07      ,  0.046     ,  0.046     ,
        -0.05357143,  0.09544643,  0.03034722,  0.15822222,  0.15822222,
         0.20944444,  0.20944444,  0.16      ,  0.165     , -0.05652778,
         0.04509091, -0.03607143, -0.066875  , -0.12706349, -0.12706349,
         0.01797619, -0.05321429, -0.08952

In [101]:
epochs = 5

In [ ]:
for e in range(epochs):
    